### Validation by aspect ratio

Instead of center cropping, sort validation images by aspect ratio. Crop batches of these images based on the closest aspect ratio

In [4]:
import argparse, os, shutil, time, warnings
from datetime import datetime
from pathlib import Path
import numpy as np
import sys

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import models
from fp16util import network_to_half, set_grad, copy_in_params

In [9]:
from torch.utils.data.sampler import Sampler
import torchvision
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
from tqdm import tqdm
import os.path
import urllib.request

In [15]:
cudnn.benchmark = True
data = '/home/paperspace/data/imagenet'
workers = 7
valdir = os.path.join(data, 'validation')
batch_size = 128

## Create Image to Aspect ratio mapping

In [16]:
# Step 1: sort images by aspect ratio
def sort_ar(valdir):
    idx2ar_file = data+'/sorted_idxar.p'
    if os.path.isfile(idx2ar_file): return pickle.load(open(idx2ar_file, 'rb'))
    print('Creating AR indexes. Please be patient this may take a couple minutes...')
    val_dataset = datasets.ImageFolder(valdir)
    sizes = [img[0].size for img in tqdm(val_dataset, total=len(val_dataset))]
    idx_ar = [(i, round(s[0]/s[1], 5)) for i,s in enumerate(sizes)]
    sorted_idxar = sorted(idx_ar, key=lambda x: x[1])
    pickle.dump(sorted_idxar, open(idx2ar_file, 'wb'))
    return sorted_idxar

# Step 2: chunk images by batch size. This way we can crop each image to the batch aspect ratio mean 
def chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

# Step 3: map image index to batch aspect ratio mean so our transform function knows where to crop
def map_idx2ar(idx_ar_sorted, batch_size):
    ar_chunks = list(chunks(idx_ar_sorted, batch_size))
    idx2ar = {}
    for chunk in ar_chunks:
        idxs, ars = list(zip(*chunk))
        mean = round(np.mean(ars), 5)
        for idx in idxs:
            idx2ar[idx] = mean
    return idx2ar

In [17]:
idx_ar_sorted = sort_ar(valdir)

In [ ]:
# TODO: create a file you can just download

## Dataset Preparation

In [18]:
class ValDataset(datasets.ImageFolder):
    def __init__(self, root, transform=None, target_transform=None, idx_transform=None):
        super().__init__(root, transform, target_transform)
        self.idx_transform = idx_transform
    def __getitem__(self, index):
        path, target = self.imgs[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.idx_transform(sample, index)
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

# Essentially a sequential sampler
class ARSampler(Sampler):
    def __init__(self, indices): self.indices = indices
    def __len__(self): return len(self.indices)
    def __iter__(self): return iter(self.indices)
    

class ArCropTfm(object):
    def __init__(self, idx2ar, target_size):
        self.idx2ar, self.target_size = idx2ar, target_size
    def __call__(self, img, idx):
        target_ar = self.idx2ar[idx]
        if target_ar < 1: 
            w = int(self.target_size/target_ar)
            size = (w//8*8, self.target_size)
        else: 
            h = int(self.target_size*target_ar)
            size = (self.target_size, h//8*8)
        return torchvision.transforms.functional.center_crop(img, size)

## Validation Function with TTA

In [28]:
def validate(val_loader, model, criterion, aug_loader=None, num_augmentations=0):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    start_time = datetime.now()

    model.eval()
    end = time.time()

    val_iter = iter(val_loader)
    aug_iters = [iter(aug_loader) for i in range(num_augmentations)]
    for i in range(len(val_loader)):
        def get_output(dl_iter):
            input,target = next(dl_iter)
            input, target = input.cuda(), target.cuda()

            # compute output
            with torch.no_grad():
                output = model(Variable(input))
                loss = criterion(output, Variable(target))
            return output, loss, input, target
        
        output,loss,input,target = get_output(val_iter)
        for aug_iter in aug_iters:
            o,l,_,_ = get_output(aug_iter)
            output.add_(o)
            loss.add_(l)
        loss.div_(num_augmentations+1)
        
        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        reduced_loss = loss.data
            
        losses.update(to_python_float(reduced_loss), input.size(0))
        top1.update(to_python_float(prec1), input.size(0))
        top5.update(to_python_float(prec5), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i%10 == 0:
            output = ('Test: [{0}/{1}]\t' \
                    + 'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t' \
                    + 'Loss {loss.val:.4f} ({loss.avg:.4f})\t' \
                    + 'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t' \
                    + 'Prec@5 {top5.val:.3f} ({top5.avg:.3f})').format(
                    i, len(val_loader), batch_time=batch_time, loss=losses,
                    top1=top1, top5=top5)
            print(output)

    time_diff = datetime.now()-start_time
    print(f'~~{epoch}\t{float(time_diff.total_seconds() / 3600.0)}\t{top5.avg:.3f}\n')
    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))

    return top1.avg

# item() is a recent addition, so this helps with backward compatibility.
def to_python_float(t):
    if hasattr(t, 'item'): return t.item()
    else: return t[0]
    
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self): self.reset()
    def reset(self): self.val = self.avg = self.sum = self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [29]:
import resnet
model = resnet.resnet50()
model = model.cuda()
model = network_to_half(model)
criterion = nn.CrossEntropyLoss().cuda()

In [ ]:
model_path = data+'/93_base_resnet50.p'
if not os.path.exists(model_path) urllib.request.urlretrieve('https://s3-us-west-2.amazonaws.com/ashaw-fastai-imagenet/93_base_resnet50.p', data+'/93_base_resnet50.p')

In [30]:
model.load_state_dict(torch.load(model_path))

### Custom Data loaders

In [31]:
def tfm_wrapper_idx(fn):
    return lambda x,idx: (fn(x),idx)
def tfm_wrapper(fn):
    return lambda x,idx: fn(x)

In [32]:
class RandomArCropTfm(object):
    def __init__(self, idx2ar, target_size):
        self.idx2ar, self.target_size = idx2ar, target_size
        self.rc = transforms.RandomCrop(0)
    def __call__(self, img, idx):
        target_ar = self.idx2ar[idx]
        if target_ar < 1: 
            w = int(self.target_size/target_ar)
            size = (w//8*8, self.target_size)
        else: 
            h = int(self.target_size*target_ar)
            size = (self.target_size, h//8*8)
        self.rc.size = size
        print(size)
        return self.rc(img)

In [33]:
class ValDataset(datasets.ImageFolder):
    def __init__(self, root, transform=None, target_transform=None):
        super().__init__(root, transform, target_transform)
    def __getitem__(self, index):
        path, target = self.imgs[index]
        sample = self.loader(path)
        if self.transform is not None:
            for tfm in self.transform:
                if isinstance(tfm, ArCropTfm) or isinstance(tfm, RandomArCropTfm): sample = tfm(sample, index)
                else: sample = tfm(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target


In [34]:
val_bs = 128
target_size = 288

idx_sorted, _ = zip(*idx_ar_sorted)
idx2ar = map_idx2ar(idx_ar_sorted, val_bs)
val_sampler_ar = ARSampler(idx_sorted)

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
tensor_tfm = [transforms.ToTensor(), normalize]



        

### Test validation with aspect ratio transforms

In [35]:
val_dataset_ar = ValDataset(valdir, [ArCropTfm(idx2ar, target_size)] + tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

validate(val_loader, model, criterion)

Test: [0/391]	Time 8.733 (8.733)	Loss 1.5127 (1.5127)	Prec@1 66.406 (66.406)	Prec@5 87.500 (87.500)
Test: [10/391]	Time 0.516 (2.234)	Loss 1.0293 (1.0240)	Prec@1 79.688 (74.077)	Prec@5 92.969 (92.543)
Test: [20/391]	Time 3.349 (1.551)	Loss 1.0449 (1.0817)	Prec@1 71.094 (72.693)	Prec@5 92.969 (92.225)
Test: [30/391]	Time 0.455 (1.568)	Loss 0.8716 (1.0731)	Prec@1 80.469 (73.236)	Prec@5 92.969 (91.935)
Test: [40/391]	Time 0.456 (1.297)	Loss 1.3506 (1.0415)	Prec@1 64.062 (73.990)	Prec@5 89.062 (92.226)
Test: [50/391]	Time 0.466 (1.132)	Loss 1.2803 (1.0966)	Prec@1 72.656 (73.131)	Prec@5 88.281 (91.376)
Test: [60/391]	Time 0.456 (1.022)	Loss 1.6611 (1.1348)	Prec@1 57.031 (72.080)	Prec@5 85.156 (90.868)
Test: [70/391]	Time 0.457 (0.943)	Loss 1.1611 (1.1243)	Prec@1 72.656 (72.359)	Prec@5 91.406 (91.032)
Test: [80/391]	Time 3.124 (0.970)	Loss 1.0908 (1.1000)	Prec@1 72.656 (72.830)	Prec@5 92.188 (91.310)
Test: [90/391]	Time 0.407 (0.978)	Loss 1.3398 (1.0952)	Prec@1 67.188 (72.888)	Prec@5 90.625 

Process Process-6:
Process Process-1:
Process Process-5:
Process Process-7:
Process Process-4:
Process Process-3:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py

  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


RuntimeError: DataLoader worker (pid 5274) exited unexpectedly with exit code 1.

### Test original validation transforms (Sorted by aspect ratio)

In [ ]:
val_tfms = [transforms.Resize(int(target_size*1.14)), transforms.CenterCrop(target_size)] + tensor_tfm
val_dataset = datasets.ImageFolder(valdir,  transforms.Compose(val_tfms))

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

validate(val_loader, model, criterion, 0, start_time)

### Test AR with size*1.14

In [ ]:
val_ar_tfms = [transforms.Resize(int(target_size*1.14)), ArCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

validate(val_loader, model, criterion, 0, start_time)

### Test TTA with original validation

In [ ]:
min_scale = 0.5
trn_tfms = [
        transforms.RandomResizedCrop(target_size, scale=(min_scale, 1.0)),
        transforms.RandomHorizontalFlip(),
    ] + tensor_tfm
aug_dataset = datasets.ImageFolder(valdir, transforms.Compose(trn_tfms))

aug_loader = torch.utils.data.DataLoader(
    aug_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)


In [ ]:
validate(val_loader, model, criterion, aug_loader=aug_loader, num_augmentations=4)

### Test TTA with size*1.14

In [ ]:
min_scale = 0.5
trn_tfms = [
        transforms.Resize(int(target_size*1.14)),
        transforms.RandomResizedCrop(target_size, scale=(min_scale, 1.0)),
        transforms.RandomHorizontalFlip(),
    ] + tensor_tfm
aug_dataset = datasets.ImageFolder(valdir, transforms.Compose(trn_tfms))

aug_loader = torch.utils.data.DataLoader(
    aug_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)
tta(val_loader, aug_loader, model, criterion, 0, start_time)

In [ ]:
min_scale = 1
trn_tfms = [
        transforms.Resize(int(target_size*1.14)),
        transforms.RandomResizedCrop(target_size, scale=(min_scale, 1.0)),
        transforms.RandomHorizontalFlip(),
    ] + tensor_tfm
aug_dataset = datasets.ImageFolder(valdir, transforms.Compose(trn_tfms))

aug_loader = torch.utils.data.DataLoader(
    aug_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)
tta(val_loader, aug_loader, model, criterion, 0, start_time)

In [ ]:
%pdb on

In [ ]:
min_scale = 0.5
aug_tfms = [
        transforms.Resize(int(target_size*1.14)),
        RandomArCropTfm(idx2ar, target_size),
        transforms.RandomHorizontalFlip(),
    ]
aug_dataset = ValDataset(valdir, aug_tfms+tensor_tfm)

aug_loader = torch.utils.data.DataLoader(
    aug_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

In [ ]:
tta(val_loader, aug_loader, model, criterion, 0, start_time)